# Comparing measured and modeled data



In [ ]:
# Install pvlib on Google Colab as this is not a standard package.
!pip install pvlib

In [ ]:
import pvlib  # library for PV and solar calculations
import pandas as pd  # library for data analysis
import matplotlib.pyplot as plt  # library for plotting
import numpy as np  # library for math and linear algebra